<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/20250327_chatgpt_peech_summarizer/blob/main/%EF%BC%92%E3%83%84%E3%83%BC%E3%83%AB%E2%98%85%E3%83%86%E3%82%B9%E3%83%88OK%E2%98%8520250328__ChatGPT_OpenAI_API%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E6%8B%A1%E5%BC%B5%E4%BC%9A%E8%A9%B1%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0_%E8%AA%BF%E3%81%B9%E3%82%82%E3%81%AE%E3%81%A8%E9%9F%B3%E5%A3%B0%E8%A6%81%E7%B4%84%EF%BC%BF%E8%A4%87%E6%95%B0URL%E5%8F%82%E7%85%A7%E3%81%A8%E8%A4%87%E6%95%B0PDF%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF%E6%A9%9F%E8%83%BD%E3%82%92%E8%BF%BD%E5%8A%A0_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03/28　テスト　音声要約ツール

In [ ]:
# 必要なライブラリをインストール
!pip install requests beautifulsoup4 PyPDF2 ipywidgets

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from IPython.display import Audio, display, HTML
import tempfile
import os
import json
import PyPDF2
from google.colab import files, userdata
import io
import ipywidgets as widgets

# APIキーをシークレットから取得
try:
    # Colabのシークレット機能からAPIキーを取得
    API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ シークレットからAPIキーを取得しました")
except Exception as e:
    print(f"⚠️ シークレットからAPIキーを取得できませんでした: {str(e)}")
    print("APIキーを手動で入力してください")
    API_KEY = input("OpenAI APIキーを入力: ")

def get_text_from_url(url):
    """
    URLからテキストコンテンツを取得する
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.encoding = response.apparent_encoding

        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text_content = '\n'.join([p.get_text().strip() for p in paragraphs])

        if len(text_content) < 100:
            text_content = soup.body.get_text(separator='\n', strip=True)

        return text_content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

def get_text_from_pdf():
    """
    アップロードされたPDFファイルからテキストを抽出する
    """
    try:
        print("PDFファイルをアップロードしてください...")
        uploaded = files.upload()

        if not uploaded:
            return "ファイルがアップロードされませんでした。"

        file_name = next(iter(uploaded))
        file_content = uploaded[file_name]

        print(f"アップロードされたファイル: {file_name}")

        # PDFからテキストを抽出
        text_content = ""
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(file_content))

        total_pages = len(pdf_reader.pages)
        print(f"PDFの総ページ数: {total_pages}")

        # 各ページからテキストを抽出
        for page_num in range(total_pages):
            page = pdf_reader.pages[page_num]
            text_content += page.extract_text() + "\n\n"

        # 処理状況を表示
        print(f"テキスト抽出完了: 総文字数 約{len(text_content)}文字")

        return text_content
    except Exception as e:
        return f"PDFの処理中にエラーが発生しました: {str(e)}"

def get_bulk_pasted_text():
    """
    大量のテキストを一度に入力できるウィジェットを表示
    """
    print("テキストを下のテキストエリアに貼り付けて「送信」ボタンをクリックしてください:")

    # テキストエリアとボタンを作成
    textarea = widgets.Textarea(
        placeholder='ここに大量のテキストを貼り付けてください...',
        layout=widgets.Layout(width='100%', height='300px')
    )

    button = widgets.Button(
        description='送信',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )

    output = widgets.Output()

    # ボタンクリック時の処理
    def on_button_clicked(b):
        with output:
            global bulk_text
            bulk_text = textarea.value
            print(f"入力完了: 総文字数 約{len(bulk_text)}文字")

    button.on_click(on_button_clicked)

    # ウィジェットを表示
    display(textarea)
    display(button)
    display(output)

    # ユーザーがボタンをクリックするのを待つ
    import time
    bulk_text = ""
    while not bulk_text:
        time.sleep(0.5)

    return bulk_text

def text_to_speech(text, voice="nova"):
    """
    テキストを音声に変換する
    """
    api_url = "https://api.openai.com/v1/audio/speech"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "tts-1",
        "voice": voice,
        "input": text
    }

    try:
        response = requests.post(api_url, headers=headers, json=data)

        if response.status_code == 200:
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as f:
                f.write(response.content)
                return f.name
        else:
            print(f"API エラー: {response.status_code}")
            print(f"エラー詳細: {response.text}")
            return None
    except Exception as e:
        print(f"リクエストエラー: {str(e)}")
        return None

def play_audio_with_speed(audio_file, speed=1.0):
    """
    音声ファイルを指定された速度で再生する
    """
    if audio_file:
        print(f"音声を再生中: {os.path.basename(audio_file)} (速度: {speed}倍)")

        # HTMLのAudioタグを使用して再生速度を制御
        audio_html = f"""
        <audio controls autoplay>
            <source src="data:audio/mp3;base64,{get_file_base64(audio_file)}" type="audio/mp3">
            お使いのブラウザは音声タグをサポートしていません。
        </audio>
        <script>
            var audios = document.getElementsByTagName('audio');
            var lastAudio = audios[audios.length-1];
            lastAudio.playbackRate = {speed};
        </script>
        """
        display(HTML(audio_html))
        return True
    return False

def get_file_base64(file_path):
    """
    ファイルをBase64エンコードする
    """
    import base64
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode('utf-8')

def get_summary_length():
    """
    要約の長さを選択する
    """
    print("\n要約の長さを選択してください")
    print("1: 超短縮（約100語/約400文字）")
    print("2: 短縮（約200語/約800文字）")
    print("3: 標準（約300語/約1200文字）")
    print("4: 詳細（約500語/約2000文字）")
    print("5: 非常に詳細（約800語/約3200文字）")
    print("6: カスタム（自分で文字数を指定）")

    choice = input("番号を選択してください (1-6) [デフォルト: 3]: ")

    length_map = {
        "1": 100,
        "2": 200,
        "3": 300,
        "4": 500,
        "5": 800
    }

    if choice == "6":
        custom_length = input("要約する単語数を入力してください（例: 400）: ")
        try:
            return int(custom_length)
        except ValueError:
            print("無効な入力です。標準の300語に設定します。")
            return 300
    else:
        return length_map.get(choice, 300)  # デフォルトは300語

def summarize_text(text, max_words=300):
    """
    OpenAI Chat APIを使ってテキストを要約する
    """
    api_url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # 文字数の目安を計算（日本語では単語数×4程度が文字数の目安）
    char_estimate = max_words * 4

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": f"あなたは与えられた文章を約{max_words}語（約{char_estimate}文字）に要約するアシスタントです。"},
            {"role": "user", "content": f"以下の文章を約{max_words}語（約{char_estimate}文字）に要約してください:\n\n{text}"}
        ]
    }

    try:
        response = requests.post(api_url, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            summary = result['choices'][0]['message']['content']

            # 実際の単語数と文字数を計算
            word_count = len(summary.split())
            char_count = len(summary)
            print(f"要約結果: 約{word_count}語、{char_count}文字")

            return summary
        else:
            print(f"要約API エラー: {response.status_code}")
            print(f"エラー詳細: {response.text}")
            return None
    except Exception as e:
        print(f"要約中にエラーが発生: {str(e)}")
        return None

def process_text_in_chunks(text, voice="nova", playback_speed=1.0, max_chunk_size=4000):
    """
    テキストを適切なサイズに分割して処理する
    """
    # テキストを適切なサイズに分割
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

    for i, chunk in enumerate(chunks):
        print(f"チャンク {i+1}/{len(chunks)} を処理中...")

        audio_file = text_to_speech(chunk, voice)
        if audio_file:
            print(f"✅ チャンク {i+1} の音声を生成しました")
            play_audio_with_speed(audio_file, playback_speed)
            # 再生後にファイルを削除
            os.remove(audio_file)
        else:
            print(f"❌ チャンク {i+1} の処理に失敗しました")

def get_voice_selection():
    """
    音声の種類を選択する
    """
    print("\n音声の種類を選択してください")
    print("1: nova (女性 - 日本語対応)")
    print("2: alloy (男性風 - 多言語)")
    print("3: echo (男性風 - 多言語)")
    print("4: fable (男性風 - 多言語)")
    print("5: onyx (男性風 - 多言語)")
    print("6: shimmer (女性風 - 多言語)")

    voice_choice = input("番号を選択してください (1-6) [デフォルト: 1]: ")

    voice_map = {
        "1": "nova",
        "2": "alloy",
        "3": "echo",
        "4": "fable",
        "5": "onyx",
        "6": "shimmer"
    }

    selected_voice = voice_map.get(voice_choice, "nova")
    print(f"選択された音声: {selected_voice}")
    return selected_voice

def get_playback_speed():
    """
    再生速度を選択する
    """
    print("\n再生速度を選択してください")
    print("1: 0.5倍 (ゆっくり)")
    print("2: 0.75倍 (やや遅い)")
    print("3: 1.0倍 (標準)")
    print("4: 1.25倍 (やや速い)")
    print("5: 1.5倍 (速い)")
    print("6: 2.0倍 (とても速い)")

    speed_choice = input("番号を選択してください (1-6) [デフォルト: 3]: ")

    speed_map = {
        "1": 0.5,
        "2": 0.75,
        "3": 1.0,
        "4": 1.25,
        "5": 1.5,
        "6": 2.0
    }

    selected_speed = speed_map.get(speed_choice, 1.0)
    print(f"選択された再生速度: {selected_speed}倍")
    return selected_speed

def get_input_source():
    """
    入力ソースを選択してテキストを取得する
    """
    print("\n入力ソースを選択してください")
    print("1: URLからテキストを取得")
    print("2: PDFファイルからテキストを抽出")
    print("3: テキストを直接入力（複数行一括入力）")

    source_choice = input("番号を選択してください (1-3): ")

    if source_choice == "1":
        # URLからテキストを取得
        url = input("\nテキストを取得するURLを入力してください: ")
        content = get_text_from_url(url)
    elif source_choice == "2":
        # PDFからテキストを抽出
        content = get_text_from_pdf()
    else:
        # テキストを一括入力
        content = get_bulk_pasted_text()

    return content

# APIが機能するか簡単なテスト
print("OpenAI APIの接続テスト中...")
test_audio = text_to_speech("This is a test", "alloy")

if test_audio and os.path.exists(test_audio):
    play_audio_with_speed(test_audio, 1.0)
    os.remove(test_audio)
    print("✅ APIテスト成功！メイン処理を開始します")

    # メイン機能選択
    print("\n実行する機能を選択してください")
    print("1: テキストの読み上げのみ")
    print("2: テキストの要約のみ")
    print("3: テキストの要約と読み上げ（要約を読み上げ）")
    print("4: テキストの要約と読み上げ（元のテキストを読み上げ）")

    feature_choice = input("番号を選択してください (1-4): ")

    # テキストの取得
    content = get_input_source()

    if content.startswith("エラー") or content.startswith("PDF"):
        print(content)
    elif not content.strip():
        print("テキストが空です。処理を終了します。")
    else:
        print(f"\n取得したテキスト（最初の300文字）: \n{content[:300]}...\n")
        print(f"全体の長さ: 約{len(content)}文字\n")

        # 要約の設定（要約機能がある場合）
        if feature_choice in ["2", "3", "4"]:
            summary_length = get_summary_length()
        else:
            summary_length = 300

        # 音声・速度設定（読み上げ機能がある場合）
        if feature_choice in ["1", "3", "4"]:
            selected_voice = get_voice_selection()
            playback_speed = get_playback_speed()
        else:
            selected_voice = "nova"
            playback_speed = 1.0

        # 選択した機能を実行
        if feature_choice == "1":
            # 読み上げのみ
            print("\nテキストを読み上げます...")
            process_text_in_chunks(content, selected_voice, playback_speed)

        elif feature_choice == "2":
            # 要約のみ
            print(f"\nテキストを約{summary_length}語に要約します...")
            summary = summarize_text(content, summary_length)

            if summary:
                print("\n要約結果:")
                print("-" * 50)
                print(summary)
                print("-" * 50)

                # 要約結果を保存するかどうか
                save_choice = input("\n要約結果をファイルに保存しますか？ (y/n): ")
                if save_choice.lower() == 'y':
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.txt', mode='w', encoding='utf-8') as f:
                        f.write(summary)
                        print(f"要約を保存しました: {f.name}")
                        # Google Colabでダウンロード
                        try:
                            from google.colab import files
                            files.download(f.name)
                        except:
                            print("ファイルをダウンロードできませんでした。")
            else:
                print("要約の作成に失敗しました。")

        elif feature_choice == "3":
            # 要約して、要約を読み上げ
            print(f"\nテキストを約{summary_length}語に要約します...")
            summary = summarize_text(content, summary_length)

            if summary:
                print("\n要約結果:")
                print("-" * 50)
                print(summary)
                print("-" * 50)

                print("\n要約を読み上げます...")
                process_text_in_chunks(summary, selected_voice, playback_speed)
            else:
                print("要約の作成に失敗しました。")

        elif feature_choice == "4":
            # 要約して、元テキストを読み上げ
            print(f"\nテキストを約{summary_length}語に要約します...")
            summary = summarize_text(content, summary_length)

            if summary:
                print("\n要約結果:")
                print("-" * 50)
                print(summary)
                print("-" * 50)

                print("\n元のテキストを読み上げます...")
                process_text_in_chunks(content, selected_voice, playback_speed)
            else:
                print("要約の作成に失敗しました。")

        print("\n処理が完了しました。")
else:
    print("❌ APIテストに失敗しました。APIキーを確認してください。")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:00
✅ シークレットからAPIキーを取得しました
OpenAI APIの接続テスト中...
音声を再生中: tmpa1hmkxrc.mp3 (速度: 1.0倍)


✅ APIテスト成功！メイン処理を開始します

実行する機能を選択してください
1: テキストの読み上げのみ
2: テキストの要約のみ
3: テキストの要約と読み上げ（要約を読み上げ）
4: テキストの要約と読み上げ（元のテキストを読み上げ）
番号を選択してください (1-4): 1

入力ソースを選択してください
1: URLからテキストを取得
2: PDFファイルからテキストを抽出
3: テキストを直接入力（複数行一括入力）
番号を選択してください (1-3): 3
テキストを下のテキストエリアに貼り付けて「送信」ボタンをクリックしてください:


Textarea(value='', layout=Layout(height='300px', width='100%'), placeholder='ここに大量のテキストを貼り付けてください...')

Button(button_style='success', description='送信', layout=Layout(width='200px'), style=ButtonStyle())

Output()

# 03/28　GPT開発補助ツール

In [ ]:
# Google Colabで使用するためのOpenAI APIを使った拡張会話システム
# URL参照とPDF読み込み機能を追加

# 必要なパッケージのインストール
!pip install -U openai openai-agents requests beautifulsoup4 PyPDF2 langchain tiktoken

# 環境変数の準備
import os
from google.colab import userdata

# 左端の鍵アイコンでOPENAI_API_KEYを設定してから実行
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from openai import OpenAI
import time
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import re
from google.colab import files
import tiktoken

# クライアントの準備
client = OpenAI()

# トークン数をカウントする関数
def count_tokens(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# テキストをチャンクに分割する関数
def split_text_into_chunks(text, max_tokens=4000):
    tokens = count_tokens(text)
    if tokens <= max_tokens:
        return [text]

    # 段落ごとに分割
    paragraphs = text.split('\n\n')
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for paragraph in paragraphs:
        paragraph_tokens = count_tokens(paragraph)

        # この段落が単体で最大トークン数を超える場合は、さらに分割
        if paragraph_tokens > max_tokens:
            words = paragraph.split()
            temp_para = ""
            for word in words:
                if count_tokens(temp_para + " " + word) <= max_tokens:
                    temp_para += " " + word
                else:
                    chunks.append(temp_para.strip())
                    temp_para = word
            if temp_para:
                chunks.append(temp_para.strip())
        # この段落を追加しても最大トークン数を超えない場合
        elif current_tokens + paragraph_tokens <= max_tokens:
            current_chunk += paragraph + "\n\n"
            current_tokens += paragraph_tokens
        # この段落を追加すると最大トークン数を超える場合
        else:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph + "\n\n"
            current_tokens = paragraph_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# URLから記事を取得するクラス
class ArticleRetriever:
    def get_article_from_url(self, url):
        """URLから記事を取得する"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # タイトルを取得
            title = soup.title.string if soup.title else "No title found"

            # 本文を取得 (いくつかの一般的なタグを試す)
            article_text = ""

            # メインコンテンツを探す
            main_content = soup.find('article') or soup.find('main') or soup.find(id=re.compile('^(content|main)'))

            if main_content:
                # 段落を取得
                paragraphs = main_content.find_all('p')
                article_text = '\n\n'.join([p.get_text() for p in paragraphs])
            else:
                # メインコンテンツが見つからない場合は、すべての段落を取得
                paragraphs = soup.find_all('p')
                article_text = '\n\n'.join([p.get_text() for p in paragraphs])

            return {
                'title': title,
                'url': url,
                'content': article_text,
                'tokens': count_tokens(article_text)
            }

        except Exception as e:
            return {
                'title': "Error retrieving article",
                'url': url,
                'content': f"Error: {str(e)}",
                'tokens': 0
            }

# PDFファイルを処理するクラス
class PDFProcessor:
    def read_pdf(self, file_path=None):
        """PDFファイルを読み込む"""
        try:
            if file_path is None:
                # Colabでファイルをアップロードする場合
                uploaded = files.upload()
                file_path = list(uploaded.keys())[0]
                pdf_file = open(file_path, 'rb')
            else:
                # ファイルパスが指定されている場合
                pdf_file = open(file_path, 'rb')

            reader = PyPDF2.PdfReader(pdf_file)
            text = ""

            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text() + "\n\n"

            pdf_file.close()

            return {
                'title': file_path,
                'content': text,
                'tokens': count_tokens(text)
            }

        except Exception as e:
            return {
                'title': "Error reading PDF",
                'content': f"Error: {str(e)}",
                'tokens': 0
            }

# 会話履歴を保持するクラス
class ConversationManager:
    def __init__(self, model="gpt-4o"):
        self.model = model
        self.messages = []
        self.previous_response_id = None
        self.use_response_id = True  # previous_response_idを使用するかのフラグ
        self.article_retriever = ArticleRetriever()
        self.pdf_processor = PDFProcessor()
        self.context = None  # 記事やPDFの内容を保持する

    def add_message(self, role, content):
        """メッセージを追加"""
        self.messages.append({"role": role, "content": content})

    def get_response(self, user_input):
        """ユーザー入力に対する応答を取得"""
        # コマンドの処理
        if user_input.startswith('/url '):
            url = user_input[5:].strip()
            article = self.article_retriever.get_article_from_url(url)

            if article['tokens'] > 0:
                self.context = {
                    'type': 'article',
                    'title': article['title'],
                    'content': article['content'],
                    'chunks': split_text_into_chunks(article['content'])
                }
                return f"記事「{article['title']}」を読み込みました。\n文字数: 約{len(article['content'])}文字\nトークン数: 約{article['tokens']}トークン\n\n質問をどうぞ。"
            else:
                return f"エラー: 記事を取得できませんでした。{article['content']}"

        elif user_input.startswith('/pdf'):
            pdf_data = self.pdf_processor.read_pdf()

            if pdf_data['tokens'] > 0:
                self.context = {
                    'type': 'pdf',
                    'title': pdf_data['title'],
                    'content': pdf_data['content'],
                    'chunks': split_text_into_chunks(pdf_data['content'])
                }
                return f"PDF「{pdf_data['title']}」を読み込みました。\n文字数: 約{len(pdf_data['content'])}文字\nトークン数: 約{pdf_data['tokens']}トークン\n\n質問をどうぞ。"
            else:
                return f"エラー: PDFを読み込めませんでした。{pdf_data['content']}"

        elif user_input.startswith('/clearcontext'):
            self.context = None
            return "コンテキストをクリアしました。"

        # 通常の会話処理
        if self.context:
            # コンテキスト（記事やPDF）が存在する場合、質問に関連する内容を追加
            context_prompt = f"以下の情報に基づいて質問に答えてください。\n\n" + \
                            f"タイトル: {self.context['title']}\n\n"

            question = user_input

            # 各チャンクに対して質問を処理
            answer = ""
            for i, chunk in enumerate(self.context['chunks']):
                # システムプロンプトを作成
                system_prompt = context_prompt + f"内容 (パート {i+1}/{len(self.context['chunks'])}):\n{chunk}\n\n" + \
                                "与えられた情報に基づいて質問に答えてください。情報が不足している場合はその旨を伝えてください。"

                try:
                    response = client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": question}
                        ],
                        temperature=0.3,
                    )

                    chunk_answer = response.choices[0].message.content
                    answer += chunk_answer + "\n\n"

                except Exception as e:
                    answer += f"エラー: {str(e)}\n\n"

            # 要約して一つの回答にまとめる（チャンクが複数ある場合）
            if len(self.context['chunks']) > 1:
                try:
                    summarize_prompt = "以下の複数の回答をまとめて、一貫性のある回答を作成してください。重複を避け、矛盾があれば解決してください。:\n\n" + answer

                    response = client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {"role": "system", "content": "与えられた複数の回答を要約し、一貫性のある回答にまとめてください。"},
                            {"role": "user", "content": summarize_prompt}
                        ],
                        temperature=0.3,
                    )

                    answer = response.choices[0].message.content

                except Exception as e:
                    answer += f"\nエラー (要約中): {str(e)}"

            # 会話履歴に追加
            self.add_message("user", user_input)
            self.add_message("assistant", answer)

            return answer

        # 通常の会話（コンテキストなし）
        if not self.messages or len(self.messages) == 0:
            # 初回の会話
            self.add_message("user", user_input)

            if self.use_response_id:
                # previous_response_idを使う方法
                response = client.responses.create(
                    model=self.model,
                    input=[{"role": "user", "content": user_input}],
                )
                self.previous_response_id = response.id

                # 応答をメッセージリストに追加
                for el in response.output:
                    self.add_message(el.role, el.content)
            else:
                # 手動で会話履歴を管理する方法
                response = client.responses.create(
                    model=self.model,
                    input=self.messages,
                    store=False
                )
                # 応答をメッセージリストに追加
                for el in response.output:
                    self.add_message(el.role, el.content)
        else:
            # 続きの会話
            self.add_message("user", user_input)

            if self.use_response_id:
                # previous_response_idを使う方法
                response = client.responses.create(
                    model=self.model,
                    previous_response_id=self.previous_response_id,
                    input=[{"role": "user", "content": user_input}],
                )
                self.previous_response_id = response.id

                # 応答をメッセージリストに追加
                for el in response.output:
                    self.add_message(el.role, el.content)
            else:
                # 手動で会話履歴を管理する方法
                response = client.responses.create(
                    model=self.model,
                    input=self.messages,
                    store=False
                )
                # 応答をメッセージリストに追加
                for el in response.output:
                    self.add_message(el.role, el.content)

        return response.output_text

    def display_history(self):
        """会話履歴を表示"""
        for msg in self.messages:
            role_display = "🧑" if msg["role"] == "user" else "🤖"
            print(f"{role_display} {msg['role']}: {msg['content']}\n")

    def switch_method(self):
        """会話状態管理方法を切り替え"""
        self.use_response_id = not self.use_response_id
        self.messages = []  # 履歴をリセット
        self.previous_response_id = None
        print(f"会話状態管理方法: {'previous_response_id' if self.use_response_id else '手動での履歴管理'}")

# 対話式インターフェース
def chat_interface():
    print("拡張ChatGPT APIとの会話を開始します。")
    print("コマンド一覧:")
    print("・exit: 会話を終了")
    print("・switch: 会話状態管理方法を切り替え")
    print("・history: 会話履歴を表示")
    print("・model: モデルを切り替え")
    print("・/url <URL>: 指定したURLの記事を取得してQAモードに切り替え")
    print("・/pdf: PDFファイルをアップロードしてQAモードに切り替え")
    print("・/clearcontext: QAモードを終了して通常会話に戻る")
    print("\n特別な機能:")
    print("・URL参照: 指定したWebページを読み込んで内容についてQAできます")
    print("・PDF処理: アップロードしたPDFの内容についてQAできます")

    # 2つのモデルを用意（切り替え可能）
    models = ["gpt-4o", "gpt-4o-mini"]
    model_idx = 0

    manager = ConversationManager(model=models[model_idx])
    print(f"現在のモデル: {models[model_idx]}")
    print(f"会話状態管理方法: {'previous_response_id' if manager.use_response_id else '手動での履歴管理'}")

    while True:
        user_input = input("\n🧑 あなた: ")

        if user_input.lower() == 'exit':
            print("会話を終了します。")
            break

        elif user_input.lower() == 'switch':
            manager.switch_method()
            continue

        elif user_input.lower() == 'model':
            model_idx = (model_idx + 1) % len(models)
            manager = ConversationManager(model=models[model_idx])
            print(f"モデルを切り替えました: {models[model_idx]}")
            continue

        elif user_input.lower() == 'history':
            manager.display_history()
            continue

        # APIからの応答を取得
        print("\n🤖 ChatGPT: ", end="")

        try:
            response_text = manager.get_response(user_input)
            print(response_text)
        except Exception as e:
            print(f"エラーが発生しました: {e}")

# インターフェースを起動
if __name__ == "__main__":
    chat_interface()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.68.2
    Uninstalling openai-1.68.2:
      Successfully uninstalled openai-1.68.2
拡張ChatGPT APIとの会話を開始します。
コマンド一覧:
・exit: 会話を終了
・switch: 会話状態管理方法を切り替え
・history: 会話履歴を表示
・model: モデルを切り替え
・/url <URL>: 指定したURLの記事を取得してQAモードに切り替え
・/pdf: PDFファイルをアップロードしてQAモードに切り替え
・/clearcontext: QAモードを終了して通常会話に戻る

特